In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 37.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 58.0 MB/s eta 0:00:00


In [ ]:
import requests
from bs4 import BeautifulSoup
from transformers import pipeline

# Initialize NER pipeline
ner = pipeline("ner", model="dbmdz/bert-large-cased-finetuned-conll03-english", tokenizer="dbmdz/bert-large-cased-finetuned-conll03-english")

# Download the text from the URL and parse it using BeautifulSoup
url = "https://www.gutenberg.org/files/164/164-h/164-h.htm"
response = requests.get(url)
content = response.text
soup = BeautifulSoup(content, "html.parser")
text = soup.get_text()

# Extract places from the entire book
places = set()
persons = set()
for chapter in text.split('\n\n'):
    ner_results = ner(chapter)
    for result in ner_results:
        if result['entity'] == 'I-LOC':
            places.add(result['word'])
        if result['entity'] == 'I-PER':
            persons.add(result['word'])

# Print the set of unique places identified in the book
print(places)
print(persons)

In [ ]:
import torch
from transformers import BertTokenizer, BertModel, pipeline
import requests
from bs4 import BeautifulSoup

# Download and load the pre-trained NER model
ner = pipeline("ner", model="dbmdz/bert-large-cased-finetuned-conll03-english", tokenizer="dbmdz/bert-large-cased-finetuned-conll03-english")

# Download and load the pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Download the text from the URL and parse it using BeautifulSoup
url = "https://www.gutenberg.org/files/164/164-h/164-h.htm"
response = requests.get(url)
content = response.text
soup = BeautifulSoup(content, "html.parser")
text = soup.get_text()

# Define function to extract places from text
def extract_places(text):
    # Tokenize text into input_ids and attention_mask tensors
    inputs = tokenizer.encode_plus(text, add_special_tokens=True, return_tensors='pt')
    
    # Feed input through BERT model to obtain contextual embeddings
    with torch.no_grad():
        outputs = model(**inputs)
        embeddings = outputs.last_hidden_state[0]
    
    # Classify each token as a place or not based on its embedding
    tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])
    places = []
    for i, token in enumerate(tokens):
        if 'place' in token.lower():  # Skip tokens with 'place' in them (e.g. "in the first place")
            continue
        if ner(token) and ner(token)[0]['entity'] == 'LOC': # use NER to classify token as a location if it is a location
            places.append(token)
    
    return places

# Extract places from the entire book
places = set()
for chapter in text.split('\n\n'):
    chapter_places = extract_places(chapter)
    for place in chapter_places:
        places.add(place)

# Print the set of unique places identified in the book
print(places)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


KeyboardInterrupt: ignored

In [ ]:
import torch
from transformers import BertTokenizer, BertModel, pipeline
import requests
from bs4 import BeautifulSoup

# Download and load the pre-trained NER model
ner = pipeline("ner", model="dbmdz/bert-large-cased-finetuned-conll03-english", tokenizer="dbmdz/bert-large-cased-finetuned-conll03-english")

# Download and load the pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Download the text from the URL and parse it using BeautifulSoup
url = "https://www.gutenberg.org/files/164/164-h/164-h.htm"
response = requests.get(url)
content = response.text
soup = BeautifulSoup(content, "html.parser")
text = soup.get_text()

def extract_entities(text, entity_type):
    # Tokenize text into input_ids and attention_mask tensors
    tokenized_text = tokenizer.tokenize(text)
    max_length = tokenizer.max_len_single_sentence - 2 # take into account [CLS] and [SEP] tokens
    tokenized_chunks = [tokenized_text[i:i+max_length] for i in range(0, len(tokenized_text), max_length)]
    inputs = [tokenizer.encode_plus(chunk, add_special_tokens=True, return_tensors='pt') for chunk in tokenized_chunks]

    # Feed input through BERT model to obtain contextual embeddings
    embeddings = []
    with torch.no_grad():
        for input in inputs:
            outputs = model(**input)
            embeddings.append(outputs.last_hidden_state)

    # Flatten embeddings into a single tensor and select only the embeddings corresponding to input tokens (i.e., exclude [CLS] and [SEP])
    embeddings = torch.cat(embeddings, dim=1)[:, 1:-1, :]

    # Use NER to extract entities of the specified type
    tokens = tokenizer.convert_ids_to_tokens(inputs[0]['input_ids'][0])
    entities = []
    for i, token in enumerate(tokens):
        if token.startswith('##'):
            continue
        if ner(token) and ner(token)[0]['entity'] == entity_type:
            entity = {
                'text': token,
                'start_char': tokenizer.convert_tokens_to_string(tokens[:i]).count(' ') + i,
                'end_char': tokenizer.convert_tokens_to_string(tokens[:i+1]).count(' ') + i,
            }
            print(entity)
            entities.append(entity)

    return entities

# Extract entities from the entire book
persons = extract_entities(text, 'PER')
places = extract_entities(text, 'LOC')

# Associate places with persons based on proximity in text
person_to_place = {}
for person, person_index in persons:
    # Find the closest place to the person in the text
    closest_place = None
    closest_distance = float('inf')
    for place, place_index in places:
        distance = abs(person_index - place_index)
        if distance < closest_distance:
            closest_distance = distance
            closest_place = place
    # Associate the closest place with the person
    if closest_place:
        person_to_place[person] = closest_place

# Print the mapping of persons to places
for person, place in person_to_place.items():
    print(person, ' -> ', place)


KeyboardInterrupt: ignored